<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









In [0]:
# Download and unzip (run only once!).
PATH_TO_LEFT_IMAGES = "leftImg8bit/train_extra/schweinfurt/"
PATH_TO_DISPARITY = "disparity/train_extra/schweinfurt/"
PATH_TO_CAMERA = "camera/train_extra/schweinfurt/"
PATH_TO_VEHICLE = "vehicle/train_extra/schweinfurt/"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip -qq leftImg8bit_trainextra.zip "$PATH_TO_LEFT_IMAGES"* -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip -qq disparity_trainextra.zip "$PATH_TO_DISPARITY"* -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip -qq camera_trainextra.zip "$PATH_TO_CAMERA"* -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip -qq vehicle_trainextra.zip "$PATH_TO_VEHICLE"* -d data
!rm vehicle_trainextra.zip

import os
PATH_TO_LEFT_IMAGES = os.path.join("data/", PATH_TO_LEFT_IMAGES)
PATH_TO_DISPARITY = os.path.join("data/", PATH_TO_DISPARITY)
PATH_TO_CAMERA = os.path.join("data/", PATH_TO_CAMERA)
PATH_TO_VEHICLE = os.path.join("data/", PATH_TO_VEHICLE)

print("Finished unzipping.")

We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
static_classes = []              

Loading an on COCO pre-trained Faster R-CNN 


Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [0]:
#@title
!pip install git+https://github.com/mapillary/seamseg.git
#!pip install wget

url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
!wget $url


These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
import numpy as np

# Calculates the difference between two coordinates in meters (x to east, y to north).
def coord_diff_to_metric_diff(d_lat, d_long, lat):
  d_y = d_lat * 110574
  d_x = d_long * 111320 * np.cos(np.radians(lat))
  return d_x, d_y

# Converts the metric difference to differences in latitude and longitude.
def metric_diff_to_coord_diff(d_x, d_y, lat):
  d_lat = d_y / 110574
  d_long = d_x / 111320 / np.cos(np.radians(lat))
  return d_lat, d_long

In [0]:
# Testing and example (taken from Johannes' map pic on WhatsApp):
# Converting coords into metrics:
d_x, d_y = coord_diff_to_metric_diff(48.370-48.369, 10.896-10.894, 48.368)
print("Delta x in meters: {}".format(d_x))
print("Delta y in meters: {}".format(d_y))

# Converting back to coords: should be the same:
d_lat, d_long = metric_diff_to_coord_diff(d_x, d_y, 48.368)
print("Difference in latitude: {}".format(d_lat))
print("Difference in longitude: {}".format(d_long))

In [0]:
# Loading results from json file stored in FEATURE_FILE 
FEATURE_FILE = "results.json"
with open(FEATURE_FILE) as json_file:
    features = json.load(json_file)

Plotting ground truth trajectory of vehicle. (Map image muss im Moment immer noch manuell gedownloaded und hinzugefügt werden)

In [0]:
# Loading ground truth data from json file and saving it as pandas array
import pandas as pd 
import os
import json
import glob
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR="data/vehicle/train_extra/schweinfurt"
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0
# create space for long at lat
gpsLatitude = np.zeros(n_images)
gpsLongitude = np.zeros(n_images)

for image_path in images_path:
  # Load json from path
  with open(image_path) as json_file:
    data = json.load(json_file)
    gpsLatitude[i] = data['gpsLatitude']
    gpsLongitude[i] = data['gpsLongitude']
    i += 1
df = pd.DataFrame({'longitude': gpsLongitude,
'latitude': gpsLatitude})
BBox = ((df.longitude.min(),   df.longitude.max(),      
         df.latitude.min(), df.latitude.max()))

longdif = df.longitude.max() - df.longitude.min()
latgdif = df.latitude.max() - df.latitude.min()
aspectr = longdif/latgdif

image = plt.imread('/content/map (4).png')

fig, ax = plt.subplots(figsize = (8,7))
ax.scatter(df.longitude, df.latitude)
ax.set_title('Plotting Trajectiory')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(image, zorder=0, extent = BBox)
ax.set_aspect(aspect= aspectr)

plt.show()


In [0]:
DATA_DIR="data/vehicle/train_extra/schweinfurt"
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0

# create space for long at lat
gpsLatitude_actual = np.zeros(n_images)
gpsLongitude_actual = np.zeros(n_images)
gpsHeading_actual = np.zeros(n_images)
gpsLatitude_delta = np.zeros(n_images)
gpsLongitude_delta = np.zeros(n_images)
gpsHeading_delta = np.zeros(n_images)


for image_path in images_path:
  # Load json from path
  with open(image_path) as json_file:
    data = json.load(json_file)
    gpsLatitude_actual[i] = data['gpsLatitude']
    gpsLongitude_actual[i] = data['gpsLongitude']
    gpsHeading_actual[i] = data['gpsHeading']
    if i > 0:
      gpsLatitude_delta[i] = gpsLatitude_actual[i] - gpsLatitude_actual[i-1]
      gpsLongitude_delta[i] = gpsLongitude_actual[i] - gpsLongitude_actual[i-1]
      gpsHeading_delta[i] = gpsHeading_actual[i] - gpsHeading_actual[i-1]
  i+=1

# Actual coordinates in x, y starting at 0, 0

gps_step_delta_X = np.zeros(n_images)
gps_total_delta_X = np.zeros(n_images)
gps_step_delta_Y = np.zeros(n_images)
gps_total_delta_Y = np.zeros(n_images)
gps_total_delta_Heading = np.zeros(n_images)

i = 0

for image_path in images_path:
  if i > 0:
    gps_step_delta_X[i], gps_step_delta_Y[i] = coord_diff_to_metric_diff(gpsLatitude_delta[i], gpsLongitude_delta[i], gpsLatitude_actual[i])
    gps_total_delta_X[i] = gps_total_delta_X[i-1] + gps_step_delta_X[i]
    gps_total_delta_Y[i] = gps_total_delta_Y[i-1] + gps_step_delta_Y[i]
    gps_total_delta_Heading[i] = gps_total_Heading[i-1] + gps_Heading_delta[i]



In [0]:
fx = 2273.82
fy = 2236.9989965173254

# Calculate real 3D position
def pixel_to_3Dposition_converter(x, depth, focal):
  x_3D = x * depth / focal
  return x_3D

# Calculate box center to get average disparity
def boundaries_to_box_center(x_1, x_2):
  x_center = (x_1 + x_2)/2
  return x_center

# Get camera focal length and baseline (To be corrected, pixels are NOT quadratic here fx =/= fy)
def camera_parameters(fx, fy):
  # Take average, which is Bullshit, but not better idea yet
  focal = (fx + fy)/2
  # Get baseline value from calibration data
  baseline = 0.222384
  return focal, baseline

# Get disparities of all points (pixels)
def disparity_catcher(x, y, i):
  DATA_DIR="data/disparity/train_extra/schweinfurt"
  im_directory = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
  n_im = len(im_directory)
  l = 0

  for im_number in im_directory:
  # Load image from path
    img = cv2.imread(im_number, 0)
  if l == i:
    global disp
    disp = img[y][x]
    print(disp)
  l += 1

  return disp

# Get disparity of center point
# def box_center_disparity(x_center, y_center):
  # disparity_box_center = pix_val_flat[1024 * 2048 * 4 - 2048 * y_center * 4 + x_center * 4]
  # return disparity_box_center

# Convert disparity to actual depth information
def disparity_to_depth(disparity_box_center):
  if disparity_box_center > 0:
    depth_center = baseline * focal / disparity_box_center
  else:
    depth_center = 100
  return depth_center

# Transformation to world frame coordinates
# _3dImage	=	cv.reprojectImageTo3D('data/disparity/train_extra/schweinfurt/schweinfurt_000000_000000_disparity.png', Q)


In [0]:
focal, baseline = camera_parameters(fx, fy)

# disparity_catcher(300, 500)

In [0]:
import math
import csv
import pandas as pd
import cv2

# Get focal length from calibration data
fx = 2273.82
fy = 2236.9989965173254

#FEATURE_FILE = 'results.csv'

# Read box coordinates from csv file
#results_df = pd.read_csv(FEATURE_FILE, sep=';')
#print(results_df.head)

#obj = results_df.to_numpy()
#print(obj[0][2][1:46])
#row = obj[0][2][1:46]
#print(features[0]['boxes'][0][0])
#print(features[0]['boxes'][1][0])

#print(features[0].keys())

class objct:
  def __init__(self, category, x1, y1, x2, y2):
    self.category = category
    self.x1 = x1
    self.y1 = y1
    self.x2 = x2
    self.y2 = y2

# class im:
  # def __init__(self, name, num_obj):
    # self.category = category
    # self.name = name
    # self.num_obj = num_obj

# obj1 = obj(2, 100, 200, 400, 300)

# print(obj1.category)
# print(obj1.x_center)

# n_images = len(images_path)

for i in images_path:
  for o in range(len(features[i]['classes'])):
    x_cent = int(boundaries_to_box_center(features[i]['boxes'][o][0], features[i]['boxes'][o][1]))
    y_cent = int(boundaries_to_box_center(features[i]['boxes'][o][2], features[i]['boxes'][o][3]))
    disp_cent = disparity_catcher(x_cent, y_cent, i)
    depth_cent = disparity_to_depth(disp_cent)
    x_3D = pixel_to_3Dposition_converter(x_cent, depth_cent, focal)
    y_3D = pixel_to_3Dposition_converter(y_cent, depth_cent, focal)
    features[i]['boxes'][o][0] = gps_total_delta_X[i] + x_3D * cos(gps_total_delta_Heading[i]) - y_3D * sin(gps_total_delta_Heading[i])
    features[i]['boxes'][o][1] = gps_total_delta_Y[i] + x_3D * sin(gps_total_delta_Heading[i]) + y_3D * cos(gps_total_delta_Heading[i])
    features[i]['boxes'][o][2] = int(0)
    features[i]['boxes'][o][3] = int(0)

#print(features[0]['boxes'][0][0:10])

   # (objct(features[i]['classes'][o], features[i]['boxes'][o][0], features[i]['boxes'][o][1], features[i]['boxes'][o][2], features[i]['boxes'][o][3]))


In [0]:
import math
import csv
import pandas as pd
import cv2

# Get focal length from calibration data
fx = 2273.82
fy = 2236.9989965173254
baseline = 0.222384


DATA_DIR="data/Julius_Results"
results_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
k = 0 # Images
t = 0 # Total object number

x1 = np.zeros(100000)
y1 = np.zeros(100000)
x2 = np.zeros(100000)
y2 = np.zeros(100000)
x_3D_rel_car = np.zeros(100000)
y_3D_rel_car = np.zeros(100000)
x_3D_abs = np.zeros(100000)
y_3D_abs = np.zeros(100000)
class_type = np.zeros(100000)

for x in results_path:
  # Load json from path
  with open(x) as json_file:
    data = json.load(json_file)
    for i in images_path:
      n_objects = int(len(data['results'][k]['classes']))
      #print(n_objects)
      #print(data['results'][k]['boxes'][0][1])
      for o in range(n_objects):
        class_type[t] = data['results'][k]['classes'][o]
        x1[t] = data['results'][k]['boxes'][o][0]
        y1[t] = data['results'][k]['boxes'][o][1]
        x2[t] = data['results'][k]['boxes'][o][2]
        y2[t] = data['results'][k]['boxes'][o][3]
        x_cent = int(boundaries_to_box_center(x1[t], x2[t]))
        y_cent = int(boundaries_to_box_center(y1[t], y2[t]))
        disp_cent = disparity_catcher(x_cent, y_cent, k)
        depth_cent = disparity_to_depth(disp_cent)
        x_3D_rel_car[t] = pixel_to_3Dposition_converter(x_cent, depth_cent, focal)
        y_3D_rel_car[t] = pixel_to_3Dposition_converter(y_cent, depth_cent, focal)
        x_3D_abs[t] = gps_total_delta_X[k] + x_3D_rel_car[t] * np.cos(gps_total_delta_Heading[k]) - y_3D_rel_car[t] * np.sin(gps_total_delta_Heading[k])
        y_3D_abs[t] = gps_total_delta_Y[k] + x_3D_rel_car[t] * np.sin(gps_total_delta_Heading[k]) + y_3D_rel_car[t] * np.cos(gps_total_delta_Heading[k])
        t += 1
      k += 1

    